In [ ]:
# Import the modules
import cv2
import numpy as np

In [ ]:
# Specify a few constants
CELL_SIZE = 20
NCLASSES = 10
TRAIN_RATIO = 0.8

In [ ]:
# Read the digits image and prepare the labels
digits_img = cv2.imread('../data/digits.png', 0)

# Display digits image
cv2.imshow('image', digits_img)
k = cv2.waitKey()

In [ ]:
# We'll slice each digit in the image to a separate small image
digits = [np.hsplit(r, digits_img.shape[1] // CELL_SIZE) 
          for r in np.vsplit(digits_img, digits_img.shape[0] // CELL_SIZE)]

# digits contains the digit image
digits = np.array(digits).reshape(-1, CELL_SIZE, CELL_SIZE)

# Total digit images
nsamples = digits.shape[0]

# labels contain the digit name
labels = np.repeat(np.arange(NCLASSES), nsamples // NCLASSES)

print('Total Digits', nsamples)

In [ ]:
index = 1232

# labels contain the digit name
# digits contains the digit image
cv2.imshow('digit is '+ str(labels[index]), digits[index])
cv2.waitKey()

In [ ]:
# Perform geometric normalization, 
# compute image moments, and align each sample
for i in range(nsamples):
    m = cv2.moments(digits[i])
    if m['mu02'] > 1e-3:
        s = m['mu11'] / m['mu02']
        M = np.float32([[1, -s, 0.5*CELL_SIZE*s], 
                        [0, 1, 0]])
        digits[i] = cv2.warpAffine(digits[i], M, (CELL_SIZE, CELL_SIZE))

In [ ]:
# Shuffle the samples
perm = np.random.permutation(nsamples)
digits = digits[perm]
labels = labels[perm]
ntrain = int(TRAIN_RATIO * nsamples)
ntest = nsamples - ntrain

In [ ]:
# Define a function for computing HOG descriptors
def calc_hog(digits):
    win_size = (20, 20)
    block_size = (10, 10)
    block_stride = (10, 10)
    cell_size = (10, 10)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    samples = []
    for d in digits: samples.append(hog.compute(d))
    return np.array(samples, np.float32)

In [ ]:
# Prepare the train and test data (features and labels)
fea_hog_train = calc_hog(digits[:ntrain])
fea_hog_test = calc_hog(digits[ntrain:])
labels_train, labels_test = labels[:ntrain], labels[ntrain:]

In [ ]:
# Create a KNN model
# K nearest neighbors is a simple algorithm that stores all items 
# and classifies new items based on a similarity measure 
# (e.g., distance functions)

K = 3
knn_model = cv2.ml.KNearest_create()
knn_model.train(fea_hog_train, cv2.ml.ROW_SAMPLE, labels_train)

In [ ]:
# Create an SVM mode
# In machine learning, support vector machines are supervised models 
# with associated learning algorithms that analyze data used for 
# classification and regression analysis

svm_model = cv2.ml.SVM_create()
svm_model.setGamma(2)
svm_model.setC(1)
svm_model.setKernel(cv2.ml.SVM_RBF)
svm_model.setType(cv2.ml.SVM_C_SVC)
svm_model.train(fea_hog_train, cv2.ml.ROW_SAMPLE, labels_train)

In [ ]:
# Define a function for evaluating the models
def eval_model(fea, labels, fpred):
    pred = fpred(fea).astype(np.int32)
    acc = (pred.T == labels).mean()*100
    
    conf_mat = np.zeros((NCLASSES, NCLASSES), np.int32)
    for c_gt, c_pred in zip(labels, pred):
        conf_mat[c_gt, c_pred] += 1
    
    return acc, conf_mat

In [ ]:
# Evaluate the KNN and SVM models
knn_acc, knn_conf_mat = eval_model(fea_hog_test, labels_test, lambda fea: knn_model.findNearest(fea, K)[1])
print('KNN accuracy (%):', knn_acc)
print('KNN confusion matrix:')
print(knn_conf_mat)

In [ ]:
svm_acc, svm_conf_mat = eval_model(fea_hog_test, labels_test, lambda fea: svm_model.predict(fea)[1])
print('SVM accuracy (%):', svm_acc)
print('SVM confusion matrix:')
print(svm_conf_mat)